In [1]:
# Import dependencies 
import pandas as pd 
import numpy as np 

In [2]:
# Create a new set of 2,000 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high = 90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high = 180.000, size=2000)

lats_lngs = zip(lats, lngs)
coordinates = list(lats_lngs)

In [3]:
# Create list of cities 

from citipy import citipy

cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    if city not in cities:
        cities.append(city)
len(cities)

760

In [4]:
# Perform an API call with the OpenWeatherMap.

import requests
from config import weather_api_key

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
city_data = []


# Print the beggining of logged data 
print("Beginning Data Retrieval")
print("-"*30)

# Create the counters 
record_count = 1
set_count = 1

# Loop through all the cities in our list usine enumerate() method 

for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    
    # Create endpoint URL with each city 
    city_url = url + "&q=" + city.replace(" ", "+")
    
    # Log the URL, record, and set numbers and the city 
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count 
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
       
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Country": city_country,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    


Beginning Data Retrieval
------------------------------
Processing Record 1 of Set 1 | kapaa
Processing Record 2 of Set 1 | mataura
Processing Record 3 of Set 1 | la ronge
Processing Record 4 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 5 of Set 1 | jamestown
Processing Record 6 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 7 of Set 1 | ciudad bolivar
Processing Record 8 of Set 1 | albany
Processing Record 9 of Set 1 | montepuez
Processing Record 10 of Set 1 | nikolskoye
Processing Record 11 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 12 of Set 1 | tasiilaq
Processing Record 13 of Set 1 | mogadishu
Processing Record 14 of Set 1 | elmira
Processing Record 15 of Set 1 | bluff
Processing Record 16 of Set 1 | rikitea
Processing Record 17 of Set 1 | bredasdorp
Processing Record 18 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 19 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 

In [5]:
len(city_data)


705

In [6]:
# Convert the array of dictionarues in city_data[] to a Pandas DataFrame 
city_data_df = pd.DataFrame(city_data)
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kapaa,US,22.0752,-159.3190,75.47,94,70,1.01,light rain
1,Mataura,NZ,-46.1927,168.8643,69.76,47,49,3.09,scattered clouds
2,La Ronge,CA,55.1001,-105.2842,38.77,68,0,7.43,clear sky
3,Jamestown,US,42.0970,-79.2353,56.30,80,81,8.99,broken clouds
4,Ciudad Bolivar,VE,8.1222,-63.5497,77.58,85,100,4.97,overcast clouds
5,Albany,US,42.6001,-73.9662,44.38,91,18,1.14,few clouds
6,Montepuez,MZ,-13.1256,38.9997,66.22,76,6,8.88,clear sky
7,Nikolskoye,RU,59.7035,30.7861,52.88,97,100,9.93,overcast clouds
8,Tasiilaq,GL,65.6145,-37.6368,28.56,69,8,8.21,clear sky
9,Mogadishu,SO,2.0371,45.3438,78.71,77,74,8.52,broken clouds


In [7]:
# Create output file (CSV)
output_data_file = "WeatherPy_Database.csv"

# Export 
city_data_df.to_csv(output_data_file, index_label="City ID")